# Домашнее задание 3
### Линейная и логистическая регрессия

Оформление дз:
Присылайте выполненное на почту `mlphystech@gmail.com` файлы .pdf и .ipynb или ссылку на свой гитхаб.

**Дедлайн: 11 апреля 23:59**

*Зуева Надежда*

ФИВТ МФТИ

Это задание будет посвящено линейной и логистической регрессии.
Мы надеемся, что вы уже знаете, что такое `kaggle`, так как сегодня нам предстоит поработать с самым *классическим* датасетом машинного обучения, а именно $-$ с Титаником!


Цель соревнования $-$ предсказать, кто из пассажиров выживет, а кто, увы, нет.

https://www.kaggle.com/c/titanic

## Теоретические вопросы
### 2 балла (по 0.5 за каждый верный ответ)
1. Что такое линейные модели классификации и регрессии? В чем их основные свойства? Где хорошо работают?
2. Что такое регуляризация и как она используется? 
3. Что такое MSE? Приведите пример.
4. Что такое градиентный спуск? Опишите базовый алгоритм работы.

P.S. просьба не копипастить из Википедии. Это все легко вычисляется и будет караться :)

1. Это модели принцип которых заключается в поиске прямой наиболее точно отражающей распределение наших величин или же прямой разбивающей наши данные на два класса(По обе стороны от прямой). Хорошо работают, когда во входных данных прослеживается линейная зависимость
2. Это добавление штрафа к линейной модели с целью борьбы с переобучением(большими весами) . L1 регуляризация зануляет(из-за штрафа) признаки, которые не влияют на результат
3. Средняя квадратичная ошибка. Сумма квадратов разностей реальной величины и полученной величины
4. Поиск экстремума через итеративный спуск(или подъем) по градиенту. Создаем начальный вектор W, далее пока градиент больше определенного eps, вычитаем из нашего набора весов, градиент от текущего набора весов * a(скорость обучения)

## Задание 1 (4 балла)
Скачайте файлы `train.csv` и `test.csv`.

Выведите первые 10 строк из `train.csv`

In [93]:
import pandas
from copy import deepcopy
import numpy as np
import pandas as pd

data = pd.read_csv("train.csv")


In [94]:
data[:10]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


Как мы видим, не все данные *

полны*. Например, это видно в столбце `Age`. 
В качестве простейшего решения мы предлагаем заменить пропущенные значения на медианные (функция `numpy.median`)
Вы можете воспользоваться нашим методом или произвести собственный анализ и заменить значения более умным способом (profit: бонусные баллы).

In [95]:
# Replace all the missing ages in the data with the median age
df = deepcopy(data)
df["Age"] = df["Age"].fillna(df["Age"].median())
df["Fare"] = df["Fare"].fillna(df["Fare"].median())

Теперь займемся **оцифровкой признаков**

Кстати, напомните, а зачем нам это нужно? :)

$*$ Чтобы все измерения выражались одинаково, то есть в числах. Тогда с ними будет легче работать$*$

Какой параметр здесь бинарный? Оцифруйте его при помощи нулей и единиц.

In [96]:
df.loc[df["Sex"] == "male", "Sex"] = 0
df.loc[df["Sex"] == "female", "Sex"] = 1
df[:5]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,S


Выведите описание нашего датасета после изменений с помощью функции `describe`.
Сколько процентов пассажиров выжило? Каков был средний возраст всех пассажиров?

In [97]:
print('Alive:', df.describe().loc['mean', 'Survived'] * 100, '%')
print ('Mean age:',df.describe().loc['mean', 'Age'])
print(df.describe())

Alive: 38.38383838383838 %
Mean age: 29.36158249158249
       PassengerId    Survived      Pclass         Sex         Age  \
count   891.000000  891.000000  891.000000  891.000000  891.000000   
mean    446.000000    0.383838    2.308642    0.352413   29.361582   
std     257.353842    0.486592    0.836071    0.477990   13.019697   
min       1.000000    0.000000    1.000000    0.000000    0.420000   
25%     223.500000    0.000000    2.000000    0.000000   22.000000   
50%     446.000000    0.000000    3.000000    0.000000   28.000000   
75%     668.500000    1.000000    3.000000    1.000000   35.000000   
max     891.000000    1.000000    3.000000    1.000000   80.000000   

            SibSp       Parch        Fare  
count  891.000000  891.000000  891.000000  
mean     0.523008    0.381594   32.204208  
std      1.102743    0.806057   49.693429  
min      0.000000    0.000000    0.000000  
25%      0.000000    0.000000    7.910400  
50%      0.000000    0.000000   14.454200  
75%   

Продолжим заполнять пропуски. Рассмотрим графу пунктов отправления `Embarked` 

Там есть типы данных:

C = Cherbourg, Q = Queenstown, S = Southampton, а также `nan`

In [98]:
print (df["Embarked"].unique())

['S' 'C' 'Q' nan]


In [99]:
import collections
print(collections.Counter(df['Embarked']))

Counter({'S': 644, 'C': 168, 'Q': 77, nan: 2})


С учетом последней ячейки заполните пропущенные значения по своему усмотрению. Объясните свой выбор, затем оцифруйте признак `Embarked` 

Например, заменив все на {0,1,2} или {12,42,10}, как хотите :)

Заменил 'nan' на 'S', потому что он наиболее часто встречается, при том в несколько раз чаще других. 

In [100]:
df["Embarked"] = df["Embarked"].fillna("S")
df.loc[df["Embarked"] == "S", "Embarked"] = 0
df.loc[df["Embarked"] == "C", "Embarked"] = 1
df.loc[df["Embarked"] == "Q", "Embarked"] = 2

Разобьем выборку на `X` и `y`, где для`y` будем строить предсказания

In [101]:
print(df.columns)

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


In [102]:
y = pd.Series(df['Survived'])

In [103]:
N_t = ['PassengerId', 'Name', 'Ticket', 'Cabin']
X = pd.DataFrame(df.loc[:, df.columns != 'Survived'])
X.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,0
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,1
2,3,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,0
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,0
4,5,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,0


Каким образом можно отобрать фичи, которые будут влиять на наш скор? Тут нам помогут ваш жизненный опыт и ` Recursive Feature Elimination` из библиотеки `Sklearn` 

In [104]:
from sklearn.feature_selection import RFE
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression, LinearRegression
model = LinearRegression(normalize=True)
rfe = RFE(model)

Проверьте, похожи ли отобранные вами фичи на те, что представленны ниже?

In [105]:
features = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]
X = X.loc[:, features]
X.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,0,22.0,1,0,7.2500,0
1,1,1,38.0,1,0,71.2833,1
2,3,1,26.0,0,0,7.9250,0
3,1,1,35.0,1,0,53.1000,0
4,3,0,35.0,0,0,8.0500,0


In [106]:
kfold = KFold(n_splits=3)
kfold.get_n_splits(X)
print(kfold)

KFold(n_splits=3, random_state=None, shuffle=False)


In [107]:
for train, test in kfold.split(X):
    print(type(list(train)))

<class 'list'>
<class 'list'>
<class 'list'>


Выполним предсказания:

In [108]:
predicts = []
print(type(model))
for train, test in kfold.split(X):
    model.fit(X.loc[train], y.loc[train])
    predicts.append(model.predict(X.loc[test]))
predicts = np.concatenate(predicts)

#print(predicts * 100)

predicts[predicts > 0.5],predicts[predicts < 0.5] = 1,0


accuracy = np.count_nonzero(df["Survived"] == predicts)/df["Survived"].count()
print ('Скор для линейной регрессии:',accuracy)


<class 'sklearn.linear_model.base.LinearRegression'>
Скор для линейной регрессии: 0.7833894500561167


## Задание 2 (2 балла)
Теперь попробуем проделать то же самое для **логистической регрессии**

http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

In [109]:
model = LogisticRegression(tol=1e-2, C=0.1)#тут можно подбирать параметры для повышения скора)

In [110]:
accus = []

Вычислим оценку точности для всех кросс-валидационных фолдов: возвращаем массив баллов из 3-х фолдов

In [111]:
predicts = []
print(type(model))
for train, test in kfold.split(X):
    model.fit(X.loc[train], y.loc[train])
    predicts.append(model.predict(X.loc[test]))
predicts = np.concatenate(predicts)

accuracy = np.count_nonzero(df["Survived"] == predicts)/df["Survived"].count()
accus.append(accuracy)
print("Скор для логистической регрессии: " + str(accuracy))

<class 'sklearn.linear_model.logistic.LogisticRegression'>
Скор для логистической регрессии: 0.797979797979798


In [112]:
print(max(accus))

0.797979797979798


###### Как бы вы объяснили прирост качества в логистической регрессии? 

Потому что именно логистическая регрессия решает задачу классификации

Сможете побить наш бейзлайн? :)

## Задание 3 (1.5 балла)
Отправьте ваше решение в тестирующую систему и посмотрите, какое место вы займете

рассмотрим `test.csv`

In [113]:
df_test = pandas.read_csv("test.csv")
print(df_test.describe())

       PassengerId      Pclass         Age       SibSp       Parch        Fare
count   418.000000  418.000000  332.000000  418.000000  418.000000  417.000000
mean   1100.500000    2.265550   30.272590    0.447368    0.392344   35.627188
std     120.810458    0.841838   14.181209    0.896760    0.981429   55.907576
min     892.000000    1.000000    0.170000    0.000000    0.000000    0.000000
25%     996.250000    1.000000   21.000000    0.000000    0.000000    7.895800
50%    1100.500000    3.000000   27.000000    0.000000    0.000000   14.454200
75%    1204.750000    3.000000   39.000000    1.000000    0.000000   31.500000
max    1309.000000    3.000000   76.000000    8.000000    9.000000  512.329200


In [114]:
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


Надо вернуть оцифрованные данные к первоначальному виду:

In [115]:
df_nw = deepcopy(df_test)
df_nw["Fare"] = df_nw["Fare"].fillna(df_nw["Fare"].median())
df_nw["Age"] = df_nw["Age"].fillna(df_nw["Age"].median())
df_nw.loc[df_nw["Sex"] == "male", "Sex"] = 0
df_nw.loc[df_nw["Sex"] == "female", "Sex"] = 1
df_nw["Embarked"] = df["Embarked"].fillna("S")
df_nw.loc[df_test["Embarked"] == "S", "Embarked"] = 0
df_nw.loc[df_test["Embarked"] == "C", "Embarked"] = 1
df_nw.loc[df_test["Embarked"] == "Q", "Embarked"] = 2
features = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]
X_pred = df_nw.loc[:, features]
X_pred.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,0,34.5,0,0,7.8292,2
1,3,1,47.0,1,0,7.0000,0
2,2,0,62.0,0,0,9.6875,2
3,3,0,27.0,0,0,8.6625,0
4,3,1,22.0,1,1,12.2875,0


In [117]:
#тут ваш код по возвращению оцифрованных параметров в первоначальный вид

model = LogisticRegression(tol=1e-4, C=0.1)
model.fit(X, y)#тут ваш код)
predicts = model.predict(X_pred)#тут ваш код)

submission = pandas.DataFrame({
    "PassengerId": df_test["PassengerId"],
    "Survived": predicts
})

submission.to_csv("submission.csv", index=False) #вот это отправляется в каггл

Можете попробовать и дальше "поиграться" с параметрами, чтобы подняться в лидерборде. Например, попробовать подобрать параметры в регрессорах. 
Удачи!
    Напоминаем, что справку по любой функции (в том числе и по регрессорам) можно получить, если с наведенным курсором на функцию нажать `Tab`+`Shift`